In [ ]:
import openpyxl as oxl, pandas as pd, numpy as np
from scipy import stats
from matplotlib import pyplot as plt

from histopath_bim_des.config import Config
from histopath_bim_des.model import Model

**Base config**

In [ ]:
path = '../assets/config_base.xlsx'
wbook = oxl.load_workbook(path, data_only=True)
config = Config.from_workbook(wbook, sim_hours=10*168, num_reps=30, runner_speed=1.2)

**Lift down**

In [ ]:
path2 = '../assets/config_lift_down.xlsx'
wbook2 = oxl.load_workbook(path2, data_only=True)
config2 = Config.from_workbook(wbook2, sim_hours=10*168, num_reps=30, runner_speed=1.2)

**Summary of runner times**

In [ ]:
df = pd.DataFrame([{x[0]: x[1] for x in config.runner_times}, {x[0]: x[1] for x in config2.runner_times}]).T
df.columns = ['Base', 'LiftDown']
df

# Simulation of Base config

In [ ]:
lab_tats = []
specimen_datas = []

for _ in range(config.num_reps):
    model_base = Model(config)
    model_base.run()
    lab_tat_new = [v['qc_end']-v['reception_start']
                   for v in model_base.specimen_data.values() if 'qc_end' in v]
    specimen_datas.append(model_base.specimen_data)
    lab_tats.append(lab_tat_new)

In [ ]:
lab_tat = np.concatenate(lab_tats)

In [ ]:
lab_tat = lab_tat/24.
np.array([np.mean(lab_tat < n) for n in range(28)])

# Simulation of lift-down config

In [ ]:
model_lift_down = Model(config2)

In [ ]:
lab_tats2 = []
specimen_datas2 = []

for _ in range(config2.num_reps):
    model_lift_down = Model(config2)
    model_lift_down.run()
    lab_tat2_new = [v['qc_end']-v['reception_start']
                   for v in model_lift_down.specimen_data.values() if 'qc_end' in v]
    lab_tats2.append(lab_tat2_new)
    specimen_datas2.append(model_lift_down.specimen_data)


In [ ]:
lab_tat2 = np.concatenate(lab_tats2)/24.
np.array([np.mean(lab_tat2 < n) for n in range(28)])

# Comparison

In [ ]:
mat1 = np.array([[np.mean(np.array(lab_tats[i])/24. < n) for n in [7,10,14,21]] for i in range(30)])
mat2 = np.array([[np.mean(np.array(lab_tats2[i])/24. < n) for n in [7,10,14,21]] for i in range(30)])

In [ ]:
mat1[:,0]

In [ ]:
means1 = np.mean(mat1, axis=0)
means1

In [ ]:
means2 = np.mean(mat2, axis=0)
means2

In [ ]:
ci1 = [
    stats.t.interval(0.95, loc=np.mean(x:=mat1[:,col]), scale=stats.sem(x), df=len(x))
    for col in range(4)
]
yerr1 = np.array([v[1] for v in ci1])-means1
yerr1

In [ ]:
ci2 = [
    stats.t.interval(0.95, loc=np.mean(x:=mat2[:,col]), scale=stats.sem(x), df=len(x))
    for col in range(4)
]
yerr2 = np.array([v[1] for v in ci2])-means2
yerr2

In [ ]:
fig = plt.figure(figsize=(5,10))
ax1, ax2 = fig.subplots(2,1)

ax1.ecdf(lab_tat)
ax1.ecdf(lab_tat2)
ax1.legend(labels=['Base config', 'Lift down'])
ax1.set_xlabel('Days')
ax1.set_title('Lab TAT, cumulative distribution')
ax1.set_ylabel('Proportion')
ax1.set_xlim(0,28)

y_df = pd.DataFrame(
    np.array([means1, means2]).T,
    index=[7,10,14,21],
    columns=['Base scenario', 'Lift down']
)
yerr_df = pd.DataFrame(
    np.array([yerr1, yerr2]).T,
    index=[7,10,14,21],
    columns=['Base scenario', 'Lift down']
)
y_df.plot(ax=ax2, kind='bar',yerr=yerr_df, capsize=5,
          xlabel='N [days]', ylabel='Proportion',
          title='Proportion of specimens completed in N days')

In [ ]:
np.mean(lab_tat)

In [ ]:
np.mean(lab_tat2)